# Getting started with CellRank

In this tutorial, you will learn how to...

- use CellRank kernels to compute a transition matrix of cellular dynamics. 
- use CellRank estimators to analyze the transition matrix, including the computation of fate probabilities, driver genes, and gene expression trends. 
- write and read CellRank kernels.  

<img src="_static/img/cellrank_overview.png" width=800  alt="CellRank is a versatile toolking to model cellular dynamics based on multi-view single-cell data, with unified analysis capabilities. ">

**Fig** | CellRank is composed of kernels, which compute a cell-cell transition matrix, and estimators, which analyze transition matrices to reveal initial & terminal states, fate probabiliteis, driver genes, and more. 

To demonstrate the appproach in this tutorial, we will use a scRNA-seq dataset of human bone marrow <cite data-cite="setty:19"></cite>. 

<div class="alert alert-info">
Important
    
If you want to run this on your own data, you will need...

- a scRNA-seq dataset for which you can compute a cell-cell transition matrix using any CellRank kernel. 

</div>

This tutorial notebook can be downloaded using the following [link](https://github.com/theislab/cellrank_notebooks/raw/master/tutorials/getting_started.ipynb).

## Import packages & data

In [1]:
import sys

if "google.colab" in sys.modules:
    !pip install -q git+https://github.com/theislab/cellrank@dev
    !pip install python-igraph

In [2]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np

scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2

Global seed set to 0


In [3]:
import warnings

warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

In [4]:
adata = cr.datasets.bone_marrow()
adata

AnnData object with n_obs × n_vars = 5780 × 27876
    obs: 'clusters', 'palantir_pseudotime', 'palantir_diff_potential'
    var: 'palantir'
    uns: 'clusters_colors', 'palantir_branch_probs_cell_types'
    obsm: 'MAGIC_imputed_data', 'X_tsne', 'palantir_branch_probs'
    layers: 'spliced', 'unspliced'

## Set up a kernel

## Kernel overview

## Combining different kernels

## Writing and reading a kernel

## Creating a new kernel - contributing to CellRank

## Set up an estimator

## Identify initial & terminal states

## Compute fate probabilities and driver genes

## Visualize expression trends

## What's next? 

In this tutorial, you learned how to use CellRank to compute a transition matrix using any precomputed pseudotime and how it can be visualized in low dimensions. The real power of CellRank comes in when you use estimators to analyze the transition matrix directly, rather than projecting it. For the next steps, we recommend...

- going through the [initial & terminal states tutorial](https://cellrank.readthedocs.io/en/stable/initial_terminal.html) to learn how to use the transition matrix to automatically identify initial and terminal states. 
- taking a look at the [full API](https://cellrank.readthedocs.io/en/stable/api.html) to learn about parameter values you can use to adapt these computations to your data. 
- exploring the vast amount of pseudotime methods to find the one that works best for your data <cite data-cite="saelens:19"></cite>. 

<div class="alert alert-info">
Important

If you encounter any bugs in the code, our if you have suggestions for new features, please open an [issue](https://github.com/theislab/cellrank/issues/new/choose). If you have a general question or something you would like to discuss with us, please post on the [scverse discourse](https://discourse.scverse.org/c/ecosytem/cellrank/40). 
    
</div>

## Package versions

In [18]:
cr.logging.print_versions()

cellrank==1.5.1+gedbc651e scanpy==1.9.3 anndata==0.8.0 numpy==1.24.2 numba==0.57.0rc1 scipy==1.10.1 pandas==1.5.3 pygpcca==1.0.4 scikit-learn==1.1.3 statsmodels==0.13.5 python-igraph==0.10.4 scvelo==0.3.0 pygam==0.8.0 matplotlib==3.7.0 seaborn==0.12.2
